In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df=pd.read_csv('/kaggle/input/loan-default-prediction/Default_Fin.csv')

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
label=[1,0]
plt.pie(df['Employed'].value_counts(), autopct='%1.1f%%',labels=label)

Employed is 70.6% and Unemployed is 29.4%

In [ ]:
label=[0,1]
plt.pie(df['Defaulted?'].value_counts(), autopct='%1.1f%%',labels=label)

Defaulted is 3.3% and not defaulted is 96.7%

Portion of bank balance

In [ ]:
def bank_balance(ex):
    if ex <= 1000:
        return 'very small'
    elif 1000<ex<=5000:
        return 'small'
    elif 5000<ex<=15000:
        return 'middle'
    elif 15000<ex<=25000:
        return 'large'
    else:
        return 'very large'
        

In [ ]:
df['balance_range']=df['Bank Balance'].apply(bank_balance)

In [ ]:
df

In [ ]:
df.groupby('balance_range').size().plot.pie(autopct='%1.1f%%')

Portion of annual salary

In [ ]:
def annual_salary(ex):
    if ex <= 100000:
        return 'very small'
    elif 100000<ex<=300000:
        return 'small'
    elif 300000<ex<=500000:
        return 'middle'
    elif 500000<ex<=700000:
        return 'large'
    else:
        return 'very large'      

In [ ]:
df['salary_range']=df['Annual Salary'].apply(annual_salary)

In [ ]:
df

In [ ]:
df.groupby('salary_range').size().plot.pie(autopct='%1.1f%%')

prediction analysis

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
x=df.drop(['Defaulted?','Index','balance_range','salary_range'],axis=1)
y=df['Defaulted?']

In [ ]:
x = StandardScaler().fit(x).transform(x)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state=10)

In [ ]:
from sklearn.linear_model import LogisticRegression
algorithm1=LogisticRegression(random_state=0)

from sklearn.tree import DecisionTreeClassifier
algorithm2=DecisionTreeClassifier(random_state=0)

from sklearn.ensemble import RandomForestClassifier
algorithm3=RandomForestClassifier(random_state=0)

from xgboost import XGBClassifier
algorithm4=XGBClassifier(random_state=0)


from catboost import CatBoostClassifier
algorithm5=CatBoostClassifier(random_state=0)

algorithms=[algorithm1,algorithm2,algorithm3,algorithm4,algorithm5]

In [ ]:
from sklearn.model_selection import StratifiedKFold
stratifiedkfold=StratifiedKFold(n_splits=3) 

from sklearn.model_selection import cross_val_score
for algorithm in algorithms:
  scores=cross_val_score(algorithm,x_train,y_train,cv=stratifiedkfold,scoring='roc_auc')
  score=scores.mean()
  name=algorithm.__class__.__name__
  print(f' average : {score:.4f}  score : {scores}  {name}')

In [ ]:
def make_cm(matrix, columns):
    n = len(columns)
    act = ['correct_data'] * n
    pred = ['prediction'] * n

    cm = pd.DataFrame(matrix, 
        columns=[pred, columns], index=[act, columns])
    return cm

In [ ]:
algorithm5.fit(x_train,y_train)
y_pred=algorithm5.predict(x_test)

from sklearn.metrics import confusion_matrix
df_matrix=make_cm(
    confusion_matrix(y_test,y_pred),['OK','Fail'])
display(df_matrix)

from sklearn.metrics import precision_recall_fscore_support
precision,recall,fscore,_=precision_recall_fscore_support(y_test,y_pred,average='binary')
print(f' precision: {precision:.4f}  recall: {recall: .4f}  Fscore:  {fscore:.4f}')

In [ ]:
import seaborn as sns
y_proba0=algorithm5.predict_proba(x_test)[:,1]

y0=y_proba0[y_test==0]
y1=y_proba0[y_test==1]

plt.figure(figsize=(6,6))
plt.title('Bank Defaulted')
sns.distplot(y1,kde=False,norm_hist=True,
             bins=50,color='b',label='fail')
sns.distplot(y0,kde=False,norm_hist=True,
             bins=50,color='k',label='OK')
plt.xlabel('thres')
plt.legend()
plt.show()

In [ ]:
def pred(algorithm5,x,thres):
  y_proba=algorithm5.predict_proba(x)
  y_proba1=y_proba[:,1]
  y_pred=(y_proba1>thres).astype(int)
  return y_pred


thres_list=np.arange(0.8,0,-0.05)

for thres in thres_list:
  y_pred=pred(algorithm5,x_test,thres)
  pred_sum=y_pred.sum()
  precision,recall,fscore,_=precision_recall_fscore_support(y_test,y_pred,average='binary')
  print(f' thres : {thres:.2f} pred_sum : {pred_sum} precision: {precision:.4f}  recall: {recall: .4f}  Fscore:  {fscore:.4f})')

What is the appropriate thres ? 

In [ ]:
y_final=pred(algorithm5,x_test,0.35)
df_matrix2=make_cm(confusion_matrix(y_test,y_final),['OK','Fail'])
display(df_matrix2)

precision,recall,fscore,_=precision_recall_fscore_support(y_test,y_final,average='binary')
print(f' precision : {precision:.4f}  recall: {recall:.4f} Fscore:{fscore:.4f}')